# Set up

In [ ]:
import os
os.environ['ISISROOT'] = '/usgs/cpkgs/anaconda3_linux/envs/isis4.2.0'

import ctypes
from ctypes.util import find_library
ctypes.CDLL(find_library('usgscsm'))

from pysis import isis

from autocnet.graph.network import NetworkCandidateGraph
from autocnet.io.db.model import Measures, Points, Images

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Morning or Evening?
morning = 'true'
kw_dict = {'true': 'morning',
           'false': 'evening'}
kw = kw_dict[morning]

In [ ]:
config_path = f'/home/ladoramkershner/projects/config_autocnet/phobos/phobos_workshop_network_generation.yml'

In [ ]:
from autocnet.config_parser import parse_config
config = parse_config(config_path)
print('db name: ', config['database']['name'])
log_dir = config['cluster']['cluster_log_dir']
print('log dir: ', log_dir)
print('ISISROOT: ', config['env']['ISISROOT'])
print('conda env: ', config['env']['conda'])

# Read in 

### First Run - File List

In [ ]:
filelist = f'/scratch/ladoramkershner/phobos.lis'

ncg = NetworkCandidateGraph()
ncg.config_from_file(config_path)
ncg.add_from_filelist(filelist, clear_db=True)

### All Subsequent Runs

In [ ]:
ncg = NetworkCandidateGraph()
ncg.config_from_file(config_path)
ncg.from_database()

In [ ]:
print('Cluster queue length:')
print('before -> ', ncg.queue_length)

In [ ]:
print('Cleaning up cluster queue:')
ncg.queue_flushdb()
print('after  -> ', ncg.queue_length)

### Visualize Graph

In [ ]:
ncg.plot()

# Place Points in Overlap

In [ ]:
# Make sure the output log directory exists
ppio_log_dir = log_dir.replace('logs', 'ppio_logs')
print(ppio_log_dir)

if not os.path.exists(ppio_log_dir):
    os.mkdir(ppio_log_dir)
    
print('Exists: ', os.path.exists(ppio_log_dir))

In [ ]:
def ns(x):
    from math import ceil
    return ceil(round(x,1)*8)

def ew(x):
    from math import ceil
    return ceil(round(x,1)*1)

distribute_points_kwargs = {'nspts_func':ns, 'ewpts_func':ew, 'method':'classic'}

njobs = ncg.apply('spatial.overlap.place_points_in_overlap', 
                  on='overlaps', 
                  walltime='00:20:00',
                  cam_type='isis',
                  distribute_points_kwargs=distribute_points_kwargs,
                  log_dir=ppio_log_dir,
                  arraychunk=100,
                  chunksize=2248)
print(njobs)


### ppio: Check queue -> reapply (if needed)

In [ ]:
redis_orphans = ncg.queue_length
print("jobs left on the queue: ", redis_orphans)

In [ ]:
# job_array = ncg.apply('spatial.overlap.place_points_in_overlap', 
#                         chunksize=redis_orphans,
#                         arraychunk=None,
#                         walltime='00:30:00',
#                         log_dir=ppio_log_dir,
#                         reapply=True)
# print(job_array)

### ppio: Write out Network

In [ ]:
path = '/scratch/ladoramkershner/kaguya/2021_mosaics/reiner_gamma/morning/control'
cnet = 'reiner_gamma_morning_ns8_ew1.net'
ncg.to_isis(os.path.join(path,cnet)) # write out

I am going to do the point registration in isis, autocnet has had work on their subpixel registration techniques and I am not sure which one to use at the moment